In [1]:
import pandas as pd
import csv
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress

In [2]:
allergy_request = pd.read_csv("data/food-allergy-analysis-Zenodo.csv")
allergy_request.head()

,SUBJECT_ID,BIRTH_YEAR,GENDER_FACTOR,RACE_FACTOR,ETHNICITY_FACTOR,PAYER_FACTOR,ATOPIC_MARCH_COHORT,AGE_START_YEARS,AGE_END_YEARS,SHELLFISH_ALG_START,...,CASHEW_ALG_END,ATOPIC_DERM_START,ATOPIC_DERM_END,ALLERGIC_RHINITIS_START,ALLERGIC_RHINITIS_END,ASTHMA_START,ASTHMA_END,FIRST_ASTHMARX,LAST_ASTHMARX,NUM_ASTHMARX
0,1,2006,S1 - Female,R1 - Black,E0 - Non-Hispanic,P1 - Medicaid,False,0.093087,3.164956,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1994,S1 - Female,R0 - White,E0 - Non-Hispanic,P0 - Non-Medicaid,False,12.232717,18.880219,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.262834,18.880219,2.0
2,3,2006,S0 - Male,R0 - White,E1 - Hispanic,P0 - Non-Medicaid,True,0.010951,6.726899,NaN,...,NaN,4.884326,NaN,3.917864,6.157426,5.127995,NaN,1.404517,6.157426,4.0
3,4,2004,S0 - Male,R4 - Unknown,E1 - Hispanic,P0 - Non-Medicaid,False,2.398357,9.111567,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2006,S1 - Female,R1 - Black,E0 - Non-Hispanic,P0 - Non-Medicaid,False,0.013689,6.193018,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
Columns_list = ['SHELLFISH_ALG_START',
       'SHELLFISH_ALG_END', 'FISH_ALG_START', 'FISH_ALG_END', 'MILK_ALG_START',
       'MILK_ALG_END', 'SOY_ALG_START', 'SOY_ALG_END', 'EGG_ALG_START',
       'EGG_ALG_END', 'WHEAT_ALG_START', 'WHEAT_ALG_END', 'PEANUT_ALG_START',
       'PEANUT_ALG_END', 'SESAME_ALG_START', 'SESAME_ALG_END', 'WALNUT_ALG_START',
       'WALNUT_ALG_END', 'PECAN_ALG_START', 'PECAN_ALG_END',
       'PISTACH_ALG_START', 'PISTACH_ALG_END', 'ALMOND_ALG_START',
       'ALMOND_ALG_END', 'BRAZIL_ALG_START', 'BRAZIL_ALG_END',
       'HAZELNUT_ALG_START', 'HAZELNUT_ALG_END', 'CASHEW_ALG_START',
       'CASHEW_ALG_END']

In [4]:
food_allergy = allergy_request.drop(columns=["FIRST_ASTHMARX","LAST_ASTHMARX","NUM_ASTHMARX","ASTHMA_END","ASTHMA_START","ALLERGIC_RHINITIS_END","ALLERGIC_RHINITIS_START","ATOPIC_DERM_END","ATOPIC_DERM_START","TREENUT_ALG_START","TREENUT_ALG_END"], axis=1)

food_allergy_clean = food_allergy.copy()

food_allergy_clean[Columns_list] = food_allergy_clean[Columns_list].astype("float")
food_allergy_clean.head()

,SUBJECT_ID,BIRTH_YEAR,GENDER_FACTOR,RACE_FACTOR,ETHNICITY_FACTOR,PAYER_FACTOR,ATOPIC_MARCH_COHORT,AGE_START_YEARS,AGE_END_YEARS,SHELLFISH_ALG_START,...,PISTACH_ALG_START,PISTACH_ALG_END,ALMOND_ALG_START,ALMOND_ALG_END,BRAZIL_ALG_START,BRAZIL_ALG_END,HAZELNUT_ALG_START,HAZELNUT_ALG_END,CASHEW_ALG_START,CASHEW_ALG_END
0,1,2006,S1 - Female,R1 - Black,E0 - Non-Hispanic,P1 - Medicaid,False,0.093087,3.164956,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1994,S1 - Female,R0 - White,E0 - Non-Hispanic,P0 - Non-Medicaid,False,12.232717,18.880219,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2006,S0 - Male,R0 - White,E1 - Hispanic,P0 - Non-Medicaid,True,0.010951,6.726899,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2004,S0 - Male,R4 - Unknown,E1 - Hispanic,P0 - Non-Medicaid,False,2.398357,9.111567,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2006,S1 - Female,R1 - Black,E0 - Non-Hispanic,P0 - Non-Medicaid,False,0.013689,6.193018,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
allergyDict = {}
 
# Adding list as value
allergyDict["Shellfish Allergy"] = ["SHELLFISH_ALG_START", "SHELLFISH_ALG_END"]
allergyDict["Fish Allergy"] = ["FISH_ALG_START", "FISH_ALG_END"]
allergyDict["Milk Allergy"] = ["MILK_ALG_START", "MILK_ALG_END"]
allergyDict["Soy Allergy"] = ["SOY_ALG_START", "SOY_ALG_END"]
allergyDict["Egg Allergy"] = ["EGG_ALG_START", "EGG_ALG_END"]
allergyDict["Wheat Allergy"] = ["WHEAT_ALG_START", "WHEAT_ALG_END"]
allergyDict["Peanut Allergy"] = ["PEANUT_ALG_START", "PEANUT_ALG_END"]
allergyDict["Sesame Allergy"] = ["SESAME_ALG_START", "SESAME_ALG_END"]
allergyDict["Walnut Allergy"] = ["WALNUT_ALG_START", "WALNUT_ALG_END"]
allergyDict["Pecan Allergy"] = ["PECAN_ALG_START", "PECAN_ALG_END"]
allergyDict["Pistachio Allergy"] = ["PISTACH_ALG_START", "PISTACH_ALG_END"]
allergyDict["Almond Allergy"] = ["ALMOND_ALG_START", "ALMOND_ALG_END"]
allergyDict["Brazil Nut Allergy"] = ["BRAZIL_ALG_START", "BRAZIL_ALG_END"]
allergyDict["Hazelnut Allergy"] = ["HAZELNUT_ALG_START", "HAZELNUT_ALG_END"]
allergyDict["Cashew Allergy"] = ["CASHEW_ALG_START", "CASHEW_ALG_END"]

In [6]:
allergyDict

{'Shellfish Allergy': ['SHELLFISH_ALG_START', 'SHELLFISH_ALG_END'],
 'Fish Allergy': ['FISH_ALG_START', 'FISH_ALG_END'],
 'Milk Allergy': ['MILK_ALG_START', 'MILK_ALG_END'],
 'Soy Allergy': ['SOY_ALG_START', 'SOY_ALG_END'],
 'Egg Allergy': ['EGG_ALG_START', 'EGG_ALG_END'],
 'Wheat Allergy': ['WHEAT_ALG_START', 'WHEAT_ALG_END'],
 'Peanut Allergy': ['PEANUT_ALG_START', 'PEANUT_ALG_END'],
 'Sesame Allergy': ['SESAME_ALG_START', 'SESAME_ALG_END'],
 'Walnut Allergy': ['WALNUT_ALG_START', 'WALNUT_ALG_END'],
 'Pecan Allergy': ['PECAN_ALG_START', 'PECAN_ALG_END'],
 'Pistachio Allergy': ['PISTACH_ALG_START', 'PISTACH_ALG_END'],
 'Almond Allergy': ['ALMOND_ALG_START', 'ALMOND_ALG_END'],
 'Brazil Nut Allergy': ['BRAZIL_ALG_START', 'BRAZIL_ALG_END'],
 'Hazelnut Allergy': ['HAZELNUT_ALG_START', 'HAZELNUT_ALG_END'],
 'Cashew Allergy': ['CASHEW_ALG_START', 'CASHEW_ALG_END']}

In [7]:
mask = food_allergy_clean.columns.str.startswith('SHELLFISH_ALG')
cols = food_allergy_clean.columns[mask]
food_allergy_clean[cols].head()

,SHELLFISH_ALG_START,SHELLFISH_ALG_END
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


In [8]:
bins = [-50, 0.35, 0.69, 3.49, 17.49, 50, 100, 1000]
group_names = ["0", "1", "2", "3", "4", "5", "6"]

for col in Columns_list:
    food_allergy_clean[col] = pd.cut(food_allergy_clean[col], bins, labels=group_names, include_lowest=True)
 
food_allergy_clean.head()

,SUBJECT_ID,BIRTH_YEAR,GENDER_FACTOR,RACE_FACTOR,ETHNICITY_FACTOR,PAYER_FACTOR,ATOPIC_MARCH_COHORT,AGE_START_YEARS,AGE_END_YEARS,SHELLFISH_ALG_START,...,PISTACH_ALG_START,PISTACH_ALG_END,ALMOND_ALG_START,ALMOND_ALG_END,BRAZIL_ALG_START,BRAZIL_ALG_END,HAZELNUT_ALG_START,HAZELNUT_ALG_END,CASHEW_ALG_START,CASHEW_ALG_END
0,1,2006,S1 - Female,R1 - Black,E0 - Non-Hispanic,P1 - Medicaid,False,0.093087,3.164956,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1994,S1 - Female,R0 - White,E0 - Non-Hispanic,P0 - Non-Medicaid,False,12.232717,18.880219,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2006,S0 - Male,R0 - White,E1 - Hispanic,P0 - Non-Medicaid,True,0.010951,6.726899,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2004,S0 - Male,R4 - Unknown,E1 - Hispanic,P0 - Non-Medicaid,False,2.398357,9.111567,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2006,S1 - Female,R1 - Black,E0 - Non-Hispanic,P0 - Non-Medicaid,False,0.013689,6.193018,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
allergyDict

{'Shellfish Allergy': ['SHELLFISH_ALG_START', 'SHELLFISH_ALG_END'],
 'Fish Allergy': ['FISH_ALG_START', 'FISH_ALG_END'],
 'Milk Allergy': ['MILK_ALG_START', 'MILK_ALG_END'],
 'Soy Allergy': ['SOY_ALG_START', 'SOY_ALG_END'],
 'Egg Allergy': ['EGG_ALG_START', 'EGG_ALG_END'],
 'Wheat Allergy': ['WHEAT_ALG_START', 'WHEAT_ALG_END'],
 'Peanut Allergy': ['PEANUT_ALG_START', 'PEANUT_ALG_END'],
 'Sesame Allergy': ['SESAME_ALG_START', 'SESAME_ALG_END'],
 'Walnut Allergy': ['WALNUT_ALG_START', 'WALNUT_ALG_END'],
 'Pecan Allergy': ['PECAN_ALG_START', 'PECAN_ALG_END'],
 'Pistachio Allergy': ['PISTACH_ALG_START', 'PISTACH_ALG_END'],
 'Almond Allergy': ['ALMOND_ALG_START', 'ALMOND_ALG_END'],
 'Brazil Nut Allergy': ['BRAZIL_ALG_START', 'BRAZIL_ALG_END'],
 'Hazelnut Allergy': ['HAZELNUT_ALG_START', 'HAZELNUT_ALG_END'],
 'Cashew Allergy': ['CASHEW_ALG_START', 'CASHEW_ALG_END']}

In [10]:
count = []
allergenTitle = []

for keys, values in allergyDict.items():
    count.append(food_allergy_clean[allergyDict[keys][0]].count())
    allergenTitle.append(keys)

totalAllergen_df = pd.DataFrame({
    "Allergy" : allergenTitle,
    "Number of Cases" : count
})

totalAllergen_df.sort_values(by="Number of Cases", ascending=False)

,Allergy,Number of Cases
6,Peanut Allergy,8653
2,Milk Allergy,7288
4,Egg Allergy,6064
0,Shellfish Allergy,5246
3,Soy Allergy,2419
1,Fish Allergy,1796
5,Wheat Allergy,1146
7,Sesame Allergy,766
8,Walnut Allergy,704
14,Cashew Allergy,561


In [11]:
allergy_name = []
allergy_severity_df = pd.DataFrame()

for keys, values in allergyDict.items():
    allergy_name.append(keys)
    df = pd.DataFrame(food_allergy_clean.groupby(allergyDict[keys][0]).size())
    df1 = df.T
    allergy_severity_df = pd.concat([allergy_severity_df, df1])

allergy_severity_df.index=allergy_name
allergy_severity_df

,0,1,2,3,4,5,6
Shellfish Allergy,11,22,1140,3812,261,0,0
Fish Allergy,4,18,660,1072,42,0,0
Milk Allergy,803,896,3455,2079,55,0,0
Soy Allergy,205,302,1040,845,27,0,0
Egg Allergy,41,349,3893,1745,36,0,0
Wheat Allergy,8,116,533,471,18,0,0
Peanut Allergy,18,193,4240,4089,113,0,0
Sesame Allergy,0,2,253,505,6,0,0
Walnut Allergy,1,4,158,519,22,0,0
Pecan Allergy,0,0,59,218,8,0,0


In [12]:
birth_years = [2006, 1994, 2004, 2005, 2003, 1999, 1997, 2001, 1990, 1993, 2002,
       2000, 1995, 1998, 1992, 1996, 1987, 1991, 1985, 1989, 1988, 2008,
       2007, 1986, 1984, 2009, 2010, 1983, 2012, 2011]
birth_years.sort()
birth_years

[1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012]

In [13]:
food_allergy_clean[food_allergy_clean["BIRTH_YEAR"]==2006]

,SUBJECT_ID,BIRTH_YEAR,GENDER_FACTOR,RACE_FACTOR,ETHNICITY_FACTOR,PAYER_FACTOR,ATOPIC_MARCH_COHORT,AGE_START_YEARS,AGE_END_YEARS,SHELLFISH_ALG_START,...,PISTACH_ALG_START,PISTACH_ALG_END,ALMOND_ALG_START,ALMOND_ALG_END,BRAZIL_ALG_START,BRAZIL_ALG_END,HAZELNUT_ALG_START,HAZELNUT_ALG_END,CASHEW_ALG_START,CASHEW_ALG_END
0,1,2006,S1 - Female,R1 - Black,E0 - Non-Hispanic,P1 - Medicaid,False,0.093087,3.164956,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2006,S0 - Male,R0 - White,E1 - Hispanic,P0 - Non-Medicaid,True,0.010951,6.726899,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2006,S1 - Female,R1 - Black,E0 - Non-Hispanic,P0 - Non-Medicaid,False,0.013689,6.193018,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,2006,S0 - Male,R0 - White,E0 - Non-Hispanic,P1 - Medicaid,False,0.087611,4.958248,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,2006,S1 - Female,R0 - White,E0 - Non-Hispanic,P0 - Non-Medicaid,False,0.010951,6.677618,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333195,333196,2006,S0 - Male,R0 - White,E0 - Non-Hispanic,P0 - Non-Medicaid,False,0.736482,7.449692,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
333196,333197,2006,S1 - Female,R1 - Black,E0 - Non-Hispanic,P1 - Medicaid,True,0.019165,6.984257,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
333197,333198,2006,S0 - Male,R0 - White,E0 - Non-Hispanic,P0 - Non-Medicaid,False,0.443532,7.405886,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
333198,333199,2006,S0 - Male,R3 - Other,E0 - Non-Hispanic,P0 - Non-Medicaid,True,0.013689,7.017112,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
birth_allergy = pd.DataFrame()


for x in birth_years:
    year_df = food_allergy_clean[food_allergy_clean["BIRTH_YEAR"]==x]
    allergy_count = []
    for keys, values in allergyDict.items():
        allergy_count.append(year_df[values[0]].count())
    birth_allergy = birth_allergy.append(pd.DataFrame([allergy_count], columns=allergyDict.keys()), ignore_index = True)
    

In [29]:
birth_allergy.index=birth_years
birth_allergy

,Shellfish Allergy,Fish Allergy,Milk Allergy,Soy Allergy,Egg Allergy,Wheat Allergy,Peanut Allergy,Sesame Allergy,Walnut Allergy,Pecan Allergy,Pistachio Allergy,Almond Allergy,Brazil Nut Allergy,Hazelnut Allergy,Cashew Allergy
1983,0,2,1,0,1,0,0,0,0,0,0,0,0,0,0
1984,2,2,1,0,1,0,0,0,0,0,0,0,0,0,0
1985,10,6,3,1,2,0,5,0,0,0,0,0,1,0,0
1986,24,5,3,1,1,1,8,0,0,0,0,0,1,0,0
1987,29,13,8,3,5,5,15,0,0,0,1,0,0,0,0
1988,60,18,17,2,10,3,43,0,2,0,0,0,0,0,2
1989,94,29,33,14,21,7,45,1,2,2,0,1,1,2,0
1990,118,32,51,21,23,11,55,2,5,2,2,2,0,1,3
1991,167,50,55,11,32,7,92,1,4,2,1,1,0,3,3
1992,199,48,60,25,48,11,133,7,13,5,1,9,0,1,2


In [36]:
medicaid_rates = pd.DataFrame()
nonmedicaid_rates = pd.DataFrame()

medicaid_mask = food_allergy_clean[food_allergy_clean["PAYER_FACTOR"]=="P1 - Medicaid"]
nonmedicaid_mask = food_allergy_clean[food_allergy_clean["PAYER_FACTOR"]=="P0 - Non-Medicaid"]

medicaid_allergy_name = []

for keys, values in allergyDict.items():
    medicaid_allergy_name.append(keys)
    df = pd.DataFrame(medicaid_mask.groupby(allergyDict[keys][0]).size())
    df1 = df.T
    medicaid_rates = pd.concat([medicaid_rates, df1])
    
nonmedicaid_allergy_name = []   
for keys, values in allergyDict.items():
    nonmedicaid_allergy_name.append(keys)
    df = pd.DataFrame(nonmedicaid_mask.groupby(allergyDict[keys][0]).size())
    df1 = df.T
    nonmedicaid_rates = pd.concat([nonmedicaid_rates, df1])
    
medicaid_rates.index=medicaid_allergy_name
nonmedicaid_rates.index=nonmedicaid_allergy_name
medicaid_rates


,0,1,2,3,4,5,6
Shellfish Allergy,3,11,471,1347,97,0,0
Fish Allergy,0,9,263,340,17,0,0
Milk Allergy,190,155,784,452,18,0,0
Soy Allergy,46,55,224,146,3,0,0
Egg Allergy,7,70,699,375,6,0,0
Wheat Allergy,0,15,107,92,1,0,0
Peanut Allergy,1,31,760,868,26,0,0
Sesame Allergy,0,1,32,76,1,0,0
Walnut Allergy,0,0,21,87,5,0,0
Pecan Allergy,0,0,7,30,5,0,0


In [38]:
nonmedicaid_rates

,0,1,2,3,4,5,6
Shellfish Allergy,8,11,669,2465,164,0,0
Fish Allergy,4,9,397,732,25,0,0
Milk Allergy,613,741,2671,1627,37,0,0
Soy Allergy,159,247,816,699,24,0,0
Egg Allergy,34,279,3194,1370,30,0,0
Wheat Allergy,8,101,426,379,17,0,0
Peanut Allergy,17,162,3480,3221,87,0,0
Sesame Allergy,0,1,221,429,5,0,0
Walnut Allergy,1,4,137,432,17,0,0
Pecan Allergy,0,0,52,188,3,0,0


In [44]:
male_rates = pd.DataFrame()
female_rates = pd.DataFrame()

male_mask = food_allergy_clean[food_allergy_clean["GENDER_FACTOR"]=="S1 - Female"]
female_mask = food_allergy_clean[food_allergy_clean["GENDER_FACTOR"]=="S0 - Male"]

male_allergy_name = []
male_count = []

for keys, values in allergyDict.items():
    male_allergy_name.append(keys)
    df = pd.DataFrame(male_mask.groupby(allergyDict[keys][0]).size())
    df1 = df.T
    male_rates = pd.concat([male_rates, df1])

female_allergy_name = [] 
female_count = []

for keys, values in allergyDict.items():
    female_allergy_name.append(keys)
    df = pd.DataFrame(female_mask.groupby(allergyDict[keys][0]).size())
    df1 = df.T
    female_rates = pd.concat([female_rates, df1])
    
female_rates.index=female_allergy_name
male_rates.index=male_allergy_name
male_rates

,0,1,2,3,4,5,6
Shellfish Allergy,4,12,524,1740,138,0,0
Fish Allergy,0,6,275,444,25,0,0
Milk Allergy,373,372,1374,883,24,0,0
Soy Allergy,93,114,413,293,14,0,0
Egg Allergy,12,114,1520,651,18,0,0
Wheat Allergy,3,44,172,171,9,0,0
Peanut Allergy,4,62,1618,1704,73,0,0
Sesame Allergy,0,1,86,181,5,0,0
Walnut Allergy,0,1,71,236,16,0,0
Pecan Allergy,0,0,24,85,6,0,0


In [42]:
female_rates

,0,1,2,3,4,5,6
Shellfish Allergy,7,10,616,2072,123,0,0
Fish Allergy,4,12,385,628,17,0,0
Milk Allergy,430,524,2081,1196,31,0,0
Soy Allergy,112,188,627,552,13,0,0
Egg Allergy,29,235,2373,1094,18,0,0
Wheat Allergy,5,72,361,300,9,0,0
Peanut Allergy,14,131,2622,2385,40,0,0
Sesame Allergy,0,1,167,324,1,0,0
Walnut Allergy,1,3,87,283,6,0,0
Pecan Allergy,0,0,35,133,2,0,0


In [ ]:
# Select columns
data = allergy_Payer_df
cols = ['SHELLFISH_ALG_START',
       'SHELLFISH_ALG_END', 'FISH_ALG_START', 'FISH_ALG_END', 'MILK_ALG_START',
       'MILK_ALG_END', 'SOY_ALG_START', 'SOY_ALG_END', 'EGG_ALG_START',
       'EGG_ALG_END', 'WHEAT_ALG_START', 'WHEAT_ALG_END', 'PEANUT_ALG_START',
       'PEANUT_ALG_END', 'SESAME_ALG_START', 'SESAME_ALG_END',
       'TREENUT_ALG_START', 'TREENUT_ALG_END', 'WALNUT_ALG_START',
       'WALNUT_ALG_END', 'PECAN_ALG_START', 'PECAN_ALG_END',
       'PISTACH_ALG_START', 'PISTACH_ALG_END', 'ALMOND_ALG_START',
       'ALMOND_ALG_END', 'BRAZIL_ALG_START', 'BRAZIL_ALG_END',
       'HAZELNUT_ALG_START', 'HAZELNUT_ALG_END', 'CASHEW_ALG_START',
       'CASHEW_ALG_END', 'ATOPIC_DERM_START', 'ATOPIC_DERM_END',
       'ALLERGIC_RHINITIS_START', 'ALLERGIC_RHINITIS_END', 'ASTHMA_START',
       'ASTHMA_END']

# Calculate averages
before = data[cols].mean(axis=0)[::2]
after = data[cols].mean(axis=0)[1::2]

In [ ]:
# Create bar graph
fig, ax = plt.subplots()
x = range(len(before))
ax.bar(x, before, width=0.4, label='Before')
ax.bar([i+0.4 for i in x], after, width=0.4, label='After')
ax.set_xticks(x)
ax.set_xticklabels(cols, rotation=90)
ax.legend()
plt.show()

In [ ]:
#Get total for all befores
allergy_df['before_total'] = allergy_df.filter(like='_START').sum(axis=1)
allergy_df['after_total'] = allergy_df.filter(like='_END').sum(axis=1)


In [ ]:
grouped_df = allergy_df.groupby('BIRTH_YEAR').agg({'before_total': 'mean', 'after_total': 'mean'})


In [ ]:
plt.bar(grouped_df.index, grouped_df['before_total'], label='Before Allergy Rate')
plt.bar(grouped_df.index, grouped_df['after_total'], bottom=grouped_df['before_total'], label='After Allergy Rate')
plt.legend()
plt.xlabel('Year')
plt.ylabel('Rate of Allergy')
plt.title('Rate of Allergy Before and After')
plt.show()